In [ ]:
# !pip install pathlib
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import tensorflow as tf
from sklearn.metrics import r2_score

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if len(device_name) > 0:
    print("Found GPU at: {}".format(device_name))
else:
    device_name = "/device:CPU:0"
    print("No GPU, using {}.".format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path=("/content/drive/MyDrive/codegnan/images2/images1")

In [ ]:
df=pd.read_csv("/content/PricesAndRatings (2).csv")
df

,Ratings,Prices
0,5.0,999
1,3.0,999
2,5.0,1749
3,1.0,1749
4,3.0,699
...,...,...
1342,2.0,399
1343,1.0,499
1344,5.0,649
1345,4.0,497


In [ ]:
import warnings
warnings.filterwarnings('ignore')
df1=pd.DataFrame({"Path":[],"Prices":[]})
Prices=[]
for i in range(len(df)):
  df1=df1.append({"Path":path+"//dress"+(str(i+1))+".png","Prices":df.iloc[i]["Prices"]},ignore_index=True)
df1

,Path,Prices
0,/content/drive/MyDrive/codegnan/images2/images...,999.0
1,/content/drive/MyDrive/codegnan/images2/images...,999.0
2,/content/drive/MyDrive/codegnan/images2/images...,1749.0
3,/content/drive/MyDrive/codegnan/images2/images...,1749.0
4,/content/drive/MyDrive/codegnan/images2/images...,699.0
...,...,...
1342,/content/drive/MyDrive/codegnan/images2/images...,399.0
1343,/content/drive/MyDrive/codegnan/images2/images...,499.0
1344,/content/drive/MyDrive/codegnan/images2/images...,649.0
1345,/content/drive/MyDrive/codegnan/images2/images...,497.0


In [ ]:
train_generator=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255,
    validation_split=0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
)
test_generator=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255,
    validation_split=0.2
)

In [ ]:
from sklearn.model_selection import train_test_split
dftrain,dftest=train_test_split(df1,random_state=25)

In [ ]:
print(len(dftrain),len(dftest))

1010 337


In [ ]:
train_images=train_generator.flow_from_dataframe(
    dataframe=dftrain,
    x_col="Path",
    y_col="Prices",
    target_size=(120,120),
    color_mode="rgb",
    class_mode="other",
    batch_size=32,
    shuffle=True,
    rescale = 1./255,
)
test_images=train_generator.flow_from_dataframe(
    dataframe=dftest,
    x_col="Path",
    y_col="Prices",
    target_size=(120,120),
    color_mode="rgb",
    class_mode="other",
    batch_size=32,
)

Found 937 validated image filenames.
Found 308 validated image filenames.


In [ ]:
train_images

In [ ]:
inputs=tf.keras.Input(shape=(120,120,3))
x=tf.keras.layers.Conv2D(filters=16,kernel_size=[3,3],activation="relu")(inputs)
x=tf.keras.layers.MaxPool2D()(x)
x=tf.keras.layers.Conv2D(filters=16,kernel_size=[3,3],activation="relu")(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)

In [ ]:
x=tf.keras.layers.Flatten()(x)
x=tf.keras.layers.Dense(64,activation="relu")(x)
x=tf.keras.layers.Dense(64,activation="relu")(x)
outputs=tf.keras.layers.Dense(1,activation="linear")(x)

In [ ]:
model=tf.keras.Model(inputs=inputs,outputs=outputs)
model.compile(
    optimizer="adam",
    loss="mse"

)

In [ ]:
history = model.fit(
    train_images,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)

Epoch 1/100
30/30 [==============================] - 6s 187ms/step - loss: 373870.9375
Epoch 2/100
30/30 [==============================] - 5s 171ms/step - loss: 370467.6562
Epoch 3/100
30/30 [==============================] - 7s 244ms/step - loss: 374432.6250
Epoch 4/100
30/30 [==============================] - 5s 171ms/step - loss: 372342.3750
Epoch 5/100
30/30 [==============================] - 7s 229ms/step - loss: 370863.9688
Epoch 6/100
30/30 [==============================] - 7s 221ms/step - loss: 369027.4375
Epoch 7/100
30/30 [==============================] - 7s 232ms/step - loss: 372493.2500
Epoch 8/100
30/30 [==============================] - 5s 178ms/step - loss: 372229.9375
Epoch 9/100
30/30 [==============================] - 5s 169ms/step - loss: 371045.6250
Epoch 10/100
30/30 [==============================] - 7s 242ms/step - loss: 369072.3125
Epoch 11/100
30/30 [==============================] - 5s 169ms/step - loss: 370770.7500


In [ ]:
model.predict(test_images)

10/10 [==============================] - 2s 167ms/step


array([[ 832.1078 ],
       [ 761.2471 ],
       [ 764.8436 ],
       [ 896.9582 ],
       [ 884.8732 ],
       [1022.07996],
       [ 687.1009 ],
       [ 887.4412 ],
       [ 823.6977 ],
       [ 771.28754],
       [ 803.1167 ],
       [1100.2396 ],
       [1087.1768 ],
       [ 614.4445 ],
       [ 991.61957],
       [1139.3463 ],
       [1121.2653 ],
       [ 666.0516 ],
       [ 976.20233],
       [ 744.8089 ],
       [ 858.61206],
       [ 782.4078 ],
       [ 678.29047],
       [ 851.9648 ],
       [ 981.1933 ],
       [ 738.8066 ],
       [ 751.25275],
       [ 903.62836],
       [ 636.9399 ],
       [1085.1454 ],
       [ 667.1508 ],
       [ 943.5949 ],
       [ 938.8051 ],
       [ 772.0855 ],
       [ 758.42096],
       [ 835.6285 ],
       [ 591.67096],
       [ 699.36053],
       [1059.0382 ],
       [1039.8184 ],
       [ 709.4429 ],
       [ 739.4837 ],
       [ 834.06744],
       [ 809.25604],
       [ 712.683  ],
       [ 988.136  ],
       [ 595.6904 ],
       [ 884.

In [ ]:
import pickle

# save the iris classification model as a pickle file
model_pkl_file = "/content/Price.pkl"

with open(model_pkl_file, 'wb') as file:
    pickle.dump(model, file)


In [ ]:
import warnings
warnings.filterwarnings('ignore')
df2=pd.DataFrame({"Path":[],"Ratings":[]})
Ratings=[]
for i in range(len(df)):
  df2=df2.append({"Path":path+"//dress"+(str(i+1))+".png","Ratings":str(df.iloc[i]["Ratings"])},ignore_index=True)
df2

,Path,Ratings
0,/content/drive/MyDrive/codegnan/images2/images...,5.0
1,/content/drive/MyDrive/codegnan/images2/images...,3.0
2,/content/drive/MyDrive/codegnan/images2/images...,5.0
3,/content/drive/MyDrive/codegnan/images2/images...,1.0
4,/content/drive/MyDrive/codegnan/images2/images...,3.0
...,...,...
1342,/content/drive/MyDrive/codegnan/images2/images...,2.0
1343,/content/drive/MyDrive/codegnan/images2/images...,1.0
1344,/content/drive/MyDrive/codegnan/images2/images...,5.0
1345,/content/drive/MyDrive/codegnan/images2/images...,4.0


In [ ]:
train_generator=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255,
    validation_split=0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
)
test_generator=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255,
    validation_split=0.2
)

In [ ]:
from sklearn.model_selection import train_test_split
dftrain,dftest=train_test_split(df2,random_state=25)

In [ ]:
train_images=train_generator.flow_from_dataframe(
    dataframe=dftrain,
    x_col="Path",
    y_col="Ratings",
    target_size=(120,120),
    color_mode="rgb",
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    rescale = 1./255,
)
test_images=train_generator.flow_from_dataframe(
    dataframe=dftest,
    x_col="Path",
    y_col="Ratings",
    target_size=(120,120),
    color_mode="rgb",
    class_mode="categorical",
    batch_size=32,
)

Found 937 validated image filenames belonging to 5 classes.
Found 308 validated image filenames belonging to 5 classes.


In [ ]:
print(df2)

                                                   Path Ratings
0     /content/drive/MyDrive/codegnan/images2/images...     5.0
1     /content/drive/MyDrive/codegnan/images2/images...     3.0
2     /content/drive/MyDrive/codegnan/images2/images...     5.0
3     /content/drive/MyDrive/codegnan/images2/images...     1.0
4     /content/drive/MyDrive/codegnan/images2/images...     3.0
...                                                 ...     ...
1342  /content/drive/MyDrive/codegnan/images2/images...     2.0
1343  /content/drive/MyDrive/codegnan/images2/images...     1.0
1344  /content/drive/MyDrive/codegnan/images2/images...     5.0
1345  /content/drive/MyDrive/codegnan/images2/images...     4.0
1346  /content/drive/MyDrive/codegnan/images2/images...     5.0

[1347 rows x 2 columns]


In [ ]:
inputs=tf.keras.Input(shape=(120,120,3))
x=tf.keras.layers.Conv2D(filters=16,kernel_size=[3,3],activation="relu")(inputs)
x=tf.keras.layers.MaxPool2D()(x)
x=tf.keras.layers.Conv2D(filters=16,kernel_size=[3,3],activation="relu")(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)

In [ ]:
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(64, activation="relu")(x)
x = tf.keras.layers.Dense(64, activation="relu")(x)
outputs = tf.keras.layers.Dense(5, activation="softmax")(x)

In [ ]:
model1=tf.keras.Model(inputs=inputs,outputs=outputs)
model1.compile(
    optimizer="adam",
    loss="categorical_crossentropy",

)

In [ ]:
history = model1.fit(
    train_images,
    epochs=10,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='accuracy',
            patience=5,
            restore_best_weights=True
        )
    ]
)


Epoch 1/10
30/30 [==============================] - ETA: 0s - loss: 1.5841

30/30 [==============================] - 5s 176ms/step - loss: 1.5841
Epoch 2/10
30/30 [==============================] - ETA: 0s - loss: 1.5933

30/30 [==============================] - 6s 211ms/step - loss: 1.5933
Epoch 3/10
30/30 [==============================] - ETA: 0s - loss: 1.5895

30/30 [==============================] - 6s 183ms/step - loss: 1.5895
Epoch 4/10
30/30 [==============================] - ETA: 0s - loss: 1.5857

30/30 [==============================] - 5s 177ms/step - loss: 1.5857
Epoch 5/10
30/30 [==============================] - ETA: 0s - loss: 1.5809

...

30/30 [==============================] - 5s 176ms/step - loss: 1.5840
Epoch 7/10
30/30 [==============================] - ETA: 0s - loss: 1.5832

30/30 [==============================] - 6s 216ms/step - loss: 1.5832
Epoch 8/10
30/30 [==============================] - ETA: 0s - loss: 1.5866

30/30 [==============================] - 6s 207ms/step - loss: 1.5866
Epoch 9/10
30/30 [==============================] - ETA: 0s - loss: 1.5841

30/30 [==============================] - 6s 197ms/step - loss: 1.5841
Epoch 10/10
30/30 [==============================] - ETA: 0s - loss: 1.5895

30/30 [==============================] - 8s 250ms/step - loss: 1.5895


In [ ]:
import pickle

# save the iris classification model as a pickle file
model_pkl_file = "Ratings.pkl"

with open(model_pkl_file, 'wb') as file:
    pickle.dump(model1, file)


In [ ]:
model.save("Price.h5")
model1.save("Ratings.h5")


In [ ]:
model1.predict(test_images)

10/10 [==============================] - 2s 174ms/step


array([[0.27774432, 0.1742222 , 0.18534471, 0.18140832, 0.18128048],
       [0.19423082, 0.19863483, 0.19806464, 0.17571788, 0.23335183],
       [0.27781406, 0.17401212, 0.18563743, 0.18190734, 0.18062904],
       ...,
       [0.19352943, 0.1991835 , 0.19815715, 0.17581363, 0.23331629],
       [0.19413424, 0.19867231, 0.19811699, 0.17582813, 0.23324837],
       [0.11054235, 0.27794984, 0.25276598, 0.05315883, 0.305583  ]],
      dtype=float32)